In [1]:
import pandas as pd
import glob
import os


In [2]:
folder_path = '/Users/nima/Downloads/archive-2'

In [ ]:
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))


dataframes = []


for file in csv_files:
    df = pd.read_csv(file)  
    dataframes.append(df)    # Append the DataFrame to the list

In [ ]:

print(dataframes[0].columns)


Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [ ]:
for i, df in enumerate(dataframes):
    print(f"File {i+1}:")
    # Count the total occurrences of each unique label value
    label_counts = df[' Label'].value_counts()
    # Get the total number of rows (total labels)
    total_labels = df[' Label'].count()

    benign_count = label_counts.get('BENIGN', 0)  
    print(f"  Total Labels: {total_labels}")
    print(f"  BENIGN Labels: {benign_count}")
    print(f"  Anomalous Labels: {total_labels - benign_count}")
    print("-" * 30)


File 1:
  Total Labels: 288602
  BENIGN Labels: 288566
  Anomalous Labels: 36
------------------------------
File 2:
  Total Labels: 529918
  BENIGN Labels: 529918
  Anomalous Labels: 0
------------------------------
File 3:
  Total Labels: 191033
  BENIGN Labels: 189067
  Anomalous Labels: 1966
------------------------------
File 4:
  Total Labels: 286467
  BENIGN Labels: 127537
  Anomalous Labels: 158930
------------------------------
File 5:
  Total Labels: 225745
  BENIGN Labels: 97718
  Anomalous Labels: 128027
------------------------------
File 6:
  Total Labels: 445909
  BENIGN Labels: 432074
  Anomalous Labels: 13835
------------------------------
File 7:
  Total Labels: 692703
  BENIGN Labels: 440031
  Anomalous Labels: 252672
------------------------------
File 8:
  Total Labels: 170366
  BENIGN Labels: 168186
  Anomalous Labels: 2180
------------------------------


In [28]:
from sklearn.preprocessing import LabelEncoder


combined_df = pd.concat(dataframes, ignore_index=True)


combined_df.columns = combined_df.columns.str.strip()


X_combined = combined_df.drop(columns=['Label'])  # Drop the 'Label' column
y_combined = combined_df['Label']  


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_combined)



In [36]:

X_combined.replace([float('inf'), float('-inf')], float('nan'), inplace=True)

X_combined.dropna(inplace=True)
#drop the corresponding y values
y_encoded = y_encoded[X_combined.index]




In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)



In [38]:
from sklearn.model_selection import train_test_split

# Split the combined data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_encoded, test_size=0.2, random_state=42)




In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9986049620210192
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    453804
           1       0.85      0.75      0.80       343
           2       1.00      1.00      1.00     25795
           3       1.00      0.99      0.99      2040
           4       1.00      1.00      1.00     46361
           5       0.99      0.99      0.99      1149
           6       1.00      1.00      1.00      1183
           7       1.00      1.00      1.00      1625
           9       1.00      0.67      0.80         6
          10       0.99      1.00      0.99     31614
          11       1.00      1.00      1.00      1187
          12       0.76      0.85      0.80       317
          13       0.50      0.20      0.29         5
          14       0.56      0.39      0.46       147

    accuracy                           1.00    565576
   macro avg       0.90      0.85      0.87    565576
weighted avg       1.00    